In [1]:
RatingsPath = r'C:\Users\ASUS\Desktop\CF\data\ml-1m\ratings.dat'
Open = open(RatingsPath, 'r')
Rating = Open.read()

raw = Rating.split("\n")

In [2]:
DATA = [[],[],[]]
mat = []
check = 1
t = []
for i in range(0, len(raw)-1):
  temp = raw[i].split('::')
  userID = int(temp[0])
  itemID = int(temp[1])
  val = int(temp[2])
  ls =[]
  # ls.append(userID-1)
  ls.append(itemID-1)
  ls.append(val)
  if(userID==check):
    t.append(ls)
  else:
    mat.append(t)
    check = userID
    t = []
  DATA[0].append(userID)
  DATA[1].append(itemID)
  DATA[2].append(val)

In [3]:
MatrixForTrain = []
MatrixForTest = []

def build(mat):
  items = 3952
  users = 6040
  for i in range(users):
    temp = []
    for j in range(items):
      temp.append(-1);
    mat.append(temp)
    # MatrixForTest.append(temp)
    # print(len(MatrixForTrain))

build(MatrixForTrain)
build(MatrixForTest)

In [4]:
print(len(MatrixForTrain))

6040


In [130]:

def function(user, train, idx, test):
    num = int(len(user)*0.8)
    for i in range(0, num):
        ls = user[i]
        train[idx][ls[0]] = ls[1]
    
    for i in range(i, len(user)):
        ls = user[i]
        test[idx][ls[0]] = ls[1]


In [137]:
for i in range(0, len(mat)):
    function(mat[i], MatrixForTrain, i, MatrixForTest)

In [6]:
preprocessDataPath = r'C:\Users\ASUS\Desktop\CF\preprocess.txt'
Open1 = open(preprocessDataPath, 'r')
preprocessRaw = Open1.read()
preProcess = preprocessRaw.split("\n")

In [7]:
def getRowForPreProcess(preProcess, idx):
    row = preProcess[idx].strip(']')
    row = row.strip('[')
    row = row.split(',')
    for j in range(len(row)):
        row[j] = float(row[j])
    
    return row

def getPreProcessMatrix(preProcess):
    matrix = []
    for idx in range(0, len(preProcess)-1):
        # print(idx)
        matrix.append(getRowForPreProcess(preProcess, idx))
    matrix.append([])
    return matrix

In [8]:
PrepocessData = getPreProcessMatrix(preProcess)
# print(preProcess[len(preProcess)-1])

In [10]:
import math

def varianceWeight(vector1, vector2):
  mean1 = 0
  mean2 = 0
  count = 0
  for i in range(len(vector1)):
    if(vector1[i]!=-1 and vector2[i]!=-1):
      mean1 = mean1 + vector1[i]
      mean2 = mean2 + vector2[i]
      count = count + 1

  if(count!=0):
    mean1 = round(float(mean1)/count, 2)
    mean2 = round(float(mean2)/count, 2)
  a2 = 0
  b2 = 0
  ab = 0
  for i in range(len(vector1)):
    if(vector1[i]!=-1 and vector2[i]!=-1):
      tempA = vector1[i] - mean1
      tempB = vector2[i] - mean2
      a2 = a2 + tempA*tempA
      b2 = b2 + tempB*tempB
      ab = tempA*tempB
  if(a2==0 or b2==0 or ab==0):
    return 0
  return round(float(ab)/(math.sqrt(a2)*math.sqrt(b2)),2)

In [12]:
def variancesimlarity(mat):
  matrix=[]
  for i in range(len(mat)):
    temp = []
    print(i)
    for j in range(i+1, len(mat)):  
      temp.append(varianceWeight(mat[i], mat[j]))
    matrix.append(temp) 
  return matrix 

In [ ]:
Variance = variancesimlarity(MatrixForTrain)

In [28]:
userMean =[]
for i in range(len(MatrixForTrain)):
  tempCount = 0
  temp = 0
  for j in range(len(MatrixForTrain[i])):
    if(MatrixForTrain[i][j]!=-1):
      temp = temp + MatrixForTrain[i][j]
      tempCount = tempCount + 1
  if(tempCount==0):
    userMean.append(1)
  else:
    userMean.append(float(temp)/tempCount)

In [88]:
def predict(matTrain, matSimilarity, userMean, item, user, K):
  similarities = {}
  for i in range(0, 943):
    if(i!=user):
      mi = min(user, i)
      ma = max(user, i)
      sim = matSimilarity[mi][ma-mi-1]
      if(mi==user):
        similarities[ma] = sim
      else:
        similarities[mi] = sim
  sortedSimilarity = sorted(similarities.items(), key=lambda x:x[1])
  simItem = []
  i = len(sortedSimilarity)-1
  while(len(simItem)!=K and i>=0):
    u = sortedSimilarity[i][0]
    it = matTrain[u][item]
    if(it!=-1):
      ls = []
      ls.append(sortedSimilarity[i][1])
      ls.append(it- userMean[u])
      simItem.append(ls)
    i = i - 1

  upper = 0
  # print(simItem)
  lower = 0
  # print(sim[0][1])
  for i in range(len(simItem)):
    lower = lower + abs(simItem[i][0])
    upper = upper + simItem[i][0]*simItem[i][1]

  if(lower==0):
    return round(userMean[user] + 0)
  ans = upper/lower
  if(ans==0):
    return  2.5
  ans = round(userMean[user] + ans)
  return ans

In [89]:
def MAE(X, Y):
  count = 0
  for i in range(len(X)):
    count = count + (abs(X[i] - Y[i]))
  return float(count)/float(len(X))

In [90]:
def NMAE(X, Y):
  val = MAE(X, Y)
  i = max(X)-min(X)
  if(i!=0):
    return val/i
  return val

In [91]:
def RMSE(X, Y):
    val = 0
    for i in range(len(X)):
        num = abs(X[i]-Y[i])
        num = num**2
        val = val + num
    print(val)
    val = val/len(X)
    print(val)
    return val**0.5


In [92]:
def predict_all(listForTest, matrixUser, matrixSimilarity, userMean, K):
  ls = []
  for i in range(len(listForTest)):
    u = listForTest[i][0]
    i = listForTest[i][1]
    temp = predict(matrixUser, matrixSimilarity,userMean, i, u, K)
    ls.append(temp)
  return ls

In [138]:
def hm(matforTest, listForTest):
  hm = []
  for i in range(len(listForTest)):
    temp = matforTest[listForTest[i][0]][listForTest[i][1]]
    hm.append(temp)
  return hm

In [141]:
def getUserItemForTest(mat):
  ls = []
  for i in range(len(mat)):
    for j in range(len(mat[i])):
      if(mat[i][j]!=-1):
        temp = []
        temp.append(i)
        temp.append(j)
        ls.append(temp)
  return ls

In [142]:
listForTest = getUserItemForTest(MatrixForTest)

In [143]:
print(len(listForTest))

306922


In [96]:
result = predict_all(listForTest, MatrixForTest, PrepocessData, userMean, 10)

In [19]:
f = open("midProcessing.txt", "a")
# f.write("Now the file has more content!")
for i in result:
    temp = str(i) +"\n"
    f.write(temp)
f.close()

In [135]:
test = hm(MatrixForTest, listForTest)

In [98]:
print(NMAE(test, result))
print(RMSE(test, result))

0.20335663458468275
380948.75
1.2411907585640651
1.1140874106478653


In [100]:
def updateFin(matForUser, listOfIndex, values, userMean):
  for i in range(len(listOfIndex)):
    matForUser[listOfIndex[i][0]][listOfIndex[i][1]] = values[i]
  
  for i in range(len(matForUser)):
    for j in range(len(matForUser[i])):
      if(matForUser[i][j]==-1):
        matForUser[i][j] = userMean[i]
      matForUser[i][j] = float(matForUser[i][j])  

In [101]:
updateFin(MatrixForTrain, listForTest, result, userMean)

In [102]:
def fillMean(matforUser, userMean):
  for i in range(len(matforUser)):
    for j in range(len(matforUser[i])):
      if(matforUser[i][j]==-1):
        matforUser[i][j] = userMean[i]
      matforUser[i][j] = float(matforUser[i][j])

In [37]:
fillMean(MatrixForTrain, userMean)

In [103]:
def getResullts(listOfIndex, R_hat):
  resultForUser= []
  for i in range(len(listOfIndex)):
    resultForUser.append(int(round(R_hat[listOfIndex[i][0]][listOfIndex[i][1]])))
    if(resultForUser[i-1]<3):
      resultForUser[i-1] = resultForUser[i-1]+0.2
    # resultForUser.append(R_hat[listOfIndex[i][0]][listOfIndex[i][1]])
  return resultForUser

In [105]:
import numpy as np
import statistics as st
from scipy.sparse.linalg import svds

def getFinalResults(matForTrain, listForTest, testU, K, userMean):
  R = [[]]
  for i in range (5):
    U, sigma, v = svds(np.array(matForTrain), 25)
    sigma = np.diag(sigma)
    R = np.dot(np.dot(U, sigma), v)
  
  for i in range(len(R)):
    for j in range(len(R[0])):
      R[i][j] = int(round(R[i][j]))
  resultOfUser = getResullts(listForTest, R)
  val = MAE(testU, resultOfUser)

  print(st.mean(testU))
  print(st.mean(resultOfUser))
  print("MAE: ", val)
  print("NMAE: ", NMAE(testU, resultOfUser))
  print("RMSE: ", RMSE(testU, resultOfUser))

In [106]:
getFinalResults(MatrixForTrain, listForTest,  test, [30,15, 20, 25], userMean)

3.5964056014231627
3.6159219606284334
MAE:  0.7514560702719513
NMAE:  0.18786401756798782
323996.8000000137
1.0556323756524906
RMSE:  1.0274397187438737


In [136]:
print(len(test))

306922


In [63]:
getFinalResults(MatrixForTrain, listForTest,  test, [20], userMean)

MAE:  0.7848962828398699
NMAE:  0.19622407070996747
298802.39915603446
0.9735450673331806
RMSE:  0.9866838740615864


In [61]:
getFinalResults(MatrixForTrain, listForTest,  test, 12, userMean)

MAE:  0.7519858465668802
NMAE:  0.18799646164172004


In [62]:
getFinalResults(MatrixForTrain, listForTest,  test, 15, userMean)

MAE:  0.7507738122389402
NMAE:  0.18769345305973506


In [63]:
getFinalResults(MatrixForTrain, listForTest,  test, 30, userMean)

MAE:  0.7502590234652452
NMAE:  0.1875647558663113


In [118]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Dropout
from keras.models import Model


In [113]:
dictionary = {
    'userId' : DATA[0],
    'movieId' : DATA[1],
    'rating' : DATA[2]
}

data = pd.DataFrame(dictionary)

In [114]:
num_users = len(data['userId'].unique())
num_items = len(data['movieId'].unique())

In [115]:
train_data, test_data = train_test_split(data, test_size=0.2)

In [123]:
def create_ncf_model(num_users, num_items, embedding_size=32, hidden_units=[128, 64, 32, 16]):
    # Input layers for user and item ids
    user_input = Input(shape=(1,), name='user_input')
    item_input = Input(shape=(1,), name='item_input')
    
    # Embedding layers for user and item ids
    user_embedding = Dense(embedding_size, name='user_embedding')(user_input)
    item_embedding = Dense(embedding_size, name='item_embedding')(item_input)
    
    # Concatenate the user and item embeddings
    concatenated = keras.layers.Concatenate()([user_embedding, item_embedding])
    
    # Add hidden layers with dropout
    for i, units in enumerate(hidden_units):
        concatenated = Dense(units, activation='relu', name=f'hidden_layer_{i}')(concatenated)
        concatenated = Dropout(0.5)(concatenated)
    
    # Output layer with sigmoid activation
    output = Dense(1, activation='sigmoid', name='output_layer')(concatenated)
    
    # Create the Keras model
    model = Model(inputs=[user_input, item_input], outputs=output)
    
    return model


In [125]:

model = create_ncf_model(num_users, num_items)
model.compile(loss='binary_crossentropy', optimizer='adam')

In [126]:
history = model.fit([train_data['userId'], train_data['movieId']], train_data['rating'], 
                    batch_size=64, epochs=10, validation_split=0.2, verbose = 0)

In [122]:
predictions = model.predict([test_data['userId'], test_data['movieId']])

6252/6252 [==============================] - 6s 995us/step


In [127]:
y_pred = []
for i in range(0, len(predictions)):
    y_pred.append(float(predictions[i][0]))